# Set-up the client

In [ ]:
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient

try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

# Get a handle to workspace
ml_client = MLClient.from_config(credential=credential)

# Retrieve an already attached Azure Machine Learning Compute.
cluster_name = "myCluster"
print(ml_client.compute.get(cluster_name))

# Data asset

### Iris dataset

In [ ]:
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes
from datetime import datetime

iris_data = Data(
    path="./data",
    type=AssetTypes.MLTABLE,
    description="Fisher's iris dataset",
    name="iris",
    version = datetime.now().strftime("%Y%m%d%H%M%S"),
)

ml_client.data.create_or_update(iris_data)

In [ ]:
import mltable

data_asset = ml_client.data.get("iris", label="latest")

tbl = mltable.load(data_asset.path)

df = tbl.to_pandas_dataframe()
df.head()

### Model specs

In [4]:
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes
from datetime import datetime

In [ ]:
archi_asset = Data(
    name="iris_architecture",
    path="./iris_model/architecture.py",
    type=AssetTypes.URI_FILE,
    description="Architecture definition for Iris classifier",
    version = datetime.now().strftime("%Y%m%d%H%M%S"),
)
ml_client.data.create_or_update(archi_asset)

In [ ]:
infer_asset = Data(
    name="iris_inference",
    path="./iris_model/inference.py",
    type=AssetTypes.URI_FILE,
    description="Inference class for Iris classifier",
    version = datetime.now().strftime("%Y%m%d%H%M%S"),
)
ml_client.data.create_or_update(infer_asset)

# Environments

## Train environment

In [ ]:
from azure.ai.ml.entities import Environment, BuildContext
from datetime import datetime

env = Environment(
    name="iris_train",
    description="PyTorch CPU training environment",
    build=BuildContext(
        path = "./environments/iris_train/",
        dockerfile_path="Dockerfile"
    ),
    version = datetime.now().strftime("%Y%m%d%H%M%S"),
)

ml_client.environments.create_or_update(env)

In [ ]:
env_check = ml_client.environments.get(
    name="iris_train",
    label = "latest"
)

print(f"✅ Environment exists: {env_check.name} v{env_check.version}")
print(f"Image URI: {env_check.image}")  # Vide si build en cours

## Unit test environment

In [ ]:
!docker build -t iris-train:latest -f environments/iris_train/Dockerfile environments/iris_train
!docker build -t iris-tests:latest -f environments/iris_tests/Dockerfile environments/iris_tests

# Unit tests

In [ ]:
pass

# Components

In [ ]:
from azure.ai.ml import load_component
from datetime import datetime

components_dir = "./components/"

In [ ]:
data_prepare_component = load_component(source=f"{components_dir}/data_prepare/data_prepare.yaml")
ml_client.components.create_or_update(
    data_prepare_component,
    version = datetime.now().strftime("%Y%m%d%H%M%S"),
)
print("✅ data_prepare registred")

In [ ]:
data_split_component = load_component(source=f"{components_dir}/data_split/data_split.yaml")
ml_client.components.create_or_update(
    data_split_component,
    version = datetime.now().strftime("%Y%m%d%H%M%S"),
)
print("✅ data_split registred")

In [ ]:
model_train_component = load_component(source=f"{components_dir}/model_train/model_train.yaml")
ml_client.components.create_or_update(
    model_train_component,
    version = datetime.now().strftime("%Y%m%d%H%M%S"),
)
print("✅ model_train registred")

In [ ]:
model_eval_component = load_component(source=f"{components_dir}/model_eval/model_eval.yaml")
ml_client.components.create_or_update(
    model_eval_component,
    version = datetime.now().strftime("%Y%m%d%H%M%S"),
)
print("✅ model_eval registred")

In [ ]:
model_save_component = load_component(source=f"{components_dir}/model_save/model_save.yaml")
ml_client.components.create_or_update(
    model_save_component,
    version = datetime.now().strftime("%Y%m%d%H%M%S"),
)
print("✅ model_save registred")

# Pipelines as components

In [ ]:
from pipelines.evaluation import iris_evaluation_pipeline
from datetime import datetime

registered_eval_pipeline = ml_client.components.create_or_update(
    iris_evaluation_pipeline,
    version=datetime.now().strftime("%Y%m%d%H%M%S"),
)

print(f"    Name: {registered_eval_pipeline.name}")
print(f"    Version: {registered_eval_pipeline.version}")

In [ ]:
from pipelines.production import iris_production_pipeline
from datetime import datetime

registered_prod_pipeline = ml_client.components.create_or_update(
    iris_production_pipeline,
    version=datetime.now().strftime("%Y%m%d%H%M%S"),
)

print(f"    Name: {registered_prod_pipeline.name}")
print(f"    Version: {registered_prod_pipeline.version}")

# Production job

In [9]:
from pipelines.production import iris_production_pipeline
from azure.ai.ml import Input
from azure.ai.ml.constants import AssetTypes

data_asset = ml_client.data.get("iris", label="latest")
archi_asset = ml_client.data.get("iris_architecture", label="latest")
infer_asset = ml_client.data.get("iris_inference", label="latest")

production_job = iris_production_pipeline(
    data = Input(
        type=AssetTypes.MLTABLE,
        path=data_asset.id,
    ),
    archi= Input(
        type=AssetTypes.URI_FILE,
        path=archi_asset.id,
    ),
    infer=Input(
        type=AssetTypes.URI_FILE,
        path=infer_asset.id,
    ),
    compute="myCluster",
)

submitted_job = ml_client.jobs.create_or_update(
    production_job,
    experiment_name="iris-classification"
)

print(f"✅ Production pipeline submitted!")
print(f"   Job name: {submitted_job.name}")
print(f"   Studio URL: {submitted_job.studio_url}")

# Optionnel : real time job monitoring
# ml_client.jobs.stream(submitted_job.name)

pathOnCompute is not a known attribute of class <class 'azure.ai.ml._restclient.v2023_04_01_preview.models._models_py3.UriFolderJobOutput'> and will be ignored


✅ Production pipeline submitted!
   Job name: helpful_animal_ql21cwwtcf
   Studio URL: https://ml.azure.com/runs/helpful_animal_ql21cwwtcf?wsid=/subscriptions/2a4d20dd-fd86-4d91-a488-68362f06e683/resourcegroups/dp100/workspaces/alex&tid=04f1b8ab-9769-4439-a61f-f14d19244457


## Recurrent execution

In [ ]:
# Get the job to schedule

all_jobs = ml_client.jobs.list()
jobs_list = [
    job for job in ml_client.jobs.list() 
    if job.experiment_name == "iris-classification" 
    and job.properties.get("runSource") != "Schedule"
]
job = jobs_list[0]
job

In [ ]:
from azure.ai.ml.entities import JobSchedule, RecurrenceTrigger

schedule = JobSchedule(
    name="iris-daily-retrain",
    trigger=RecurrenceTrigger(
        frequency="hour",  # or "day", "week", "month"
        interval=1
    ),
    create_job=job.name
)

ml_client.schedules.begin_create_or_update(schedule=schedule)

## Custom parameters

In [ ]:
all_jobs = ml_client.jobs.list()
jobs_list = [
    job for job in ml_client.jobs.list() 
    if job.experiment_name == "iris-classification" 
    and job.properties.get("runSource") != "Schedule"
]
job = jobs_list[0]
job

In [ ]:
from azure.ai.ml import Input
from datetime import datetime
from azure.ai.ml.constants import AssetTypes

# job =  ml_client.jobs.get(name = "frank_airport_nl42g9wj33")

production_pipeline = ml_client.components.get(
    name="iris_production_pipeline",
    label="latest",
)

custom_job = ml_client.jobs.create_or_update(
    production_pipeline(
        data=Input(
            type=AssetTypes.MLTABLE,
            path=job.inputs.data.path
        ),
        archi=Input(
            type=AssetTypes.URI_FILE,
            path=job.inputs.archi.path
        ),
        compute="myCluster",
        lr=0.001,
        epochs=20,
    ),
    experiment_name=job.experiment_name,
    display_name = "20epochs_" + datetime.now().strftime("%Y%m%d%H%M%S")
)


# Deploy the model as a real-time endpoint

### Collect the model

In [10]:
all_jobs = ml_client.jobs.list()
jobs_list = [
    job.name for job in ml_client.jobs.list() 
    if job.experiment_name == "iris-classification" 
   # and job.properties.get("runSource") != "Schedule"
]
jobs_list

['helpful_animal_ql21cwwtcf',
 'eager_bucket_794sj5ktqb',
 'iris-daily-retrain-08584369864271870488966060766CU11',
 'iris-daily-retrain-08584369900278820444886051514CU04',
 'iris-daily-retrain-08584369936267978446581133173CU09',
 'hungry_kitten_zqxhdzhytv',
 'sleepy_jelly_n94z16r4v8']

In [12]:
all_models = ml_client.models.list(name = "iris-model-ready")
models_list = [model for model in all_models]
models_list

[Model({'job_name': '22d22530-397f-4827-bec3-c4adadb883f5', 'intellectual_property': None, 'system_metadata': None, 'is_anonymous': False, 'auto_increment_version': False, 'auto_delete_setting': None, 'name': 'iris-model-ready', 'description': None, 'tags': {}, 'properties': {'flavors.python_function': '{\n  "streamable": false,\n  "cloudpickle_version": "2.2.1",\n  "python_model": "python_model.pkl",\n  "artifacts": {\n    "model": {\n      "path": "artifacts/model.pth",\n      "uri": "/mnt/azureml/cr/j/e7df3a251a21461db53878f3bde30dbe/cap/data-capability/wd/INPUT_model/model.pth"\n    },\n    "scaler": {\n      "path": "artifacts/scaler.pkl",\n      "uri": "/mnt/azureml/cr/j/e7df3a251a21461db53878f3bde30dbe/cap/data-capability/wd/INPUT_scaler/scaler.pkl"\n    },\n    "mapping": {\n      "path": "artifacts/mapping.json",\n      "uri": "/mnt/azureml/cr/j/e7df3a251a21461db53878f3bde30dbe/cap/data-capability/wd/INPUT_mapping/mapping.json"\n    },\n    "archi": {\n      "path": "artifacts

In [13]:
model = models_list[0]
job_child_name = model.properties.get('azureml.artifactPrefix').split('/')[1][5:]
job_child = ml_client.jobs.get(name=job_child_name)
job_child.tags["azureml.pipeline"]

/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/mlflow/__init__.py:41: UserWarning: Versions of mlflow (3.1.1) and mlflow-skinny (2.22.1) are different. This may lead to unexpected behavior. Please install the same version of both packages.
  mlflow.mismatch._check_version_mismatch()


'helpful_animal_ql21cwwtcf'

In [14]:
print(f"Model: {model.name} - version: {model.version}")

Model: iris-model-ready - version: 1


### Also collect the environment

In [15]:
environment = ml_client.environments.get(name="iris_train", label="latest")

### Debug (bash)

In [ ]:
# az account set --subscription "123-456-789"

# az provider register --namespace Microsoft.MachineLearningServices
# az provider register --namespace Microsoft.PolicyInsights
# az provider register --namespace Microsoft.Cdn
# az provider register --namespace Microsoft.ApiManagement

# for p in Microsoft.MachineLearningServices \
#          Microsoft.PolicyInsights \
#          Microsoft.Cdn \
#          Microsoft.MachineLearningServices \
#          Microsoft.ApiManagement; do
#   echo "==== $p ===="
#   az provider show -n "$p" --query "{namespace:namespace, state:registrationState}"
# done


### RTI service to a managed endpoint

In [17]:
from azure.ai.ml.entities import ManagedOnlineEndpoint
# from azure.ai.ml.entities import KubernetesOnlineEndpoint, KubernetesOnlineDeployment # AKS

# Endpoint creation
endpoint = ManagedOnlineEndpoint(
    name="iris-managed-endpoint",
    description="Iris classification endpoint",
    auth_mode="key",
)
ml_client.online_endpoints.begin_create_or_update(endpoint).wait()
print("✅ endpoint registred")

✅ endpoint registred


In [18]:
from azure.ai.ml.entities import ManagedOnlineDeployment, CodeConfiguration

# Model deployment
deployment = ManagedOnlineDeployment(
    name="iris-deployment",
    endpoint_name="iris-managed-endpoint",
    model=model,
    environment=environment,
    code_configuration=CodeConfiguration(
        code="./scoring_script",
        scoring_script="scoring_script.py"
    ),
    instance_count=1,
    instance_type="Standard_D2a_v4"
)
ml_client.online_deployments.begin_create_or_update(deployment).wait()
print("✅ model deployed")

Check: endpoint iris-managed-endpoint exists
Uploading scoring_script (0.0 MBs): 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████

............................................................................................................................................✅ model deployed
